# Flow Rate Data

Graph of flow rate data from San Diego River at Fashion Valley (USGS 11023000)

In [1]:
import pandas as pd
import requests as req
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool

Get JSON file of the water data from USGS website

Documentation for formatting for json retrieval from USGS can be found at

<https://help.waterdata.usgs.gov/faq/automated-retrievals>

In [2]:
r = req.get('http://waterservices.usgs.gov/nwis/iv/?sites=11023000&period=P365D&parameterCd=00060&format=json')
json_data = r.json()

In [3]:
parsed_data = []
for i in json_data['value']['timeSeries']:
    for j in i['values']:
        for k in j['value']:
            parsed_data.append(k)

cfs_df = pd.DataFrame(parsed_data)

In [4]:
# Convert strings into operable types
for idx, row in cfs_df.iterrows():
    
    # Remove ':' from timezone formatting
    # USGS JSON formatting: 2017-06-17T21:45:00.000-07:00
    # Expected formatting:  2017-06-17T21:45:00.000-0700
    datetime_stripped_str = row.dateTime[:-3] + row.dateTime[-2:]
    
    cfs_df.iloc[idx].dateTime = pd.to_datetime(datetime_stripped_str)
    cfs_df.iloc[idx].value = float(row.value)

In [8]:
x = cfs_df.dateTime
y = cfs_df.value

hover = HoverTool(tooltips = [('Date', '@x{%F}'),
                              ('Flow Rate', '@y')],
                  formatters = {'x' : 'datetime'}, 
                  mode = 'vline')

plot = figure(title = 'Flow Rate at San Diego River at Fashion Valley',
              x_axis_type ='datetime',
              x_axis_label='date',
              y_axis_label='flow rate (cfs)',
              tools = [hover],
              plot_width = 800,
              plot_height = 500)

plot.line(x, y)

output_notebook()
show(plot)

Loading BokehJS ...